In [67]:
from coredotfinance.data import KrxReader
import pandas as pd
krx = KrxReader()
company_df = krx.listed_company()
dataframe = krx.read_date('2022-07-01', kor=True)

In [72]:
listed_company_df = pd.merge(company_df, dataframe, on='종목코드', how='inner')
filtered_column = listed_company_df[['종목명_x', '시장구분_x', '업종명', '상장주식수_x', '액면가', '자본금','업종코드', '시가총액']]
print(filtered_column)

       종목명_x  시장구분_x                      업종명    상장주식수_x     액면가  \
0         3S  KOSDAQ                 전자부품 제조업   46271513   500.0   
1     AJ네트웍스   KOSPI          산업용 기계 및 장비 임대업   46822295  1000.0   
2      AK홀딩스   KOSPI                   기타 금융업   13247561  5000.0   
3     APS홀딩스  KOSDAQ                   기타 금융업   20394221   500.0   
4      AP시스템  KOSDAQ            특수 목적용 기계 제조업   15281421   500.0   
...      ...     ...                      ...        ...     ...   
2501  흥국에프엔비  KOSDAQ          비알코올음료 및 얼음 제조업   39997771   100.0   
2502    흥국화재   KOSPI                      보험업   64242645  5000.0   
2503    흥아해운   KOSPI                   해상 운송업  240424899   500.0   
2504      희림  KOSDAQ  건축기술 엔지니어링 및 관련 기술 서비스업   13922475   500.0   
2505      힘스  KOSDAQ            특수 목적용 기계 제조업   11312236   500.0   

               자본금    업종코드          시가총액  
0      23135756500  032602  135112817960  
1      46822295000  147603  302003802750  
2      66237805000  116409  210636219900  
3      1019